In [20]:
import pandas as pd
from collections import defaultdict

In [2]:
CSV = '../../data/tira-annotated-metadata.csv'

In [33]:
df = pd.read_csv(CSV, keep_default_na=False)
df.head()

,,Filename,Duration,RA work complete,Annotation in ELAN,Annotations done(ish)?,Files uploaded to Kwaras?,Elicitation leader,Transcriber,Present,Category,Content,num_utterances,utterance_len_total,utterance_len_avg
0,7/31/2020,HH07312020-Audacity,0:54:11,,Sharon,yes,9/1/2020,Peter,Peter,"Nina, Sharon, Peter",Paradigms,"conjugation of 'fetch water' in perfective, im...",136,0:04:21,0:00:01
1,8/7/2020,HH08072020,1:10:50,,Nina,yes,9/2/2020,Nina,Sharon,"Nina, Sharon",Paradigms,"minimal pair for consonant quantity (one/any),...",288,0:10:24,0:00:02
2,8/14/2020,HH08142020,1:15:46,,Sharon,yes,9/3/2020,Nina/Sharon,Peter,"Nina, Sharon, Peter",Paradigms,"Good morning, subj/obj forms of water, worm, b...",108,0:03:46,0:00:02
3,8/21/2020,HH08212020-1,0:25:06,,Sharon,yes,9/10/2020,Sharon,Sharon,"Nina, Sharon",Phrases,"Subj/obj forms of person names (Kalu, Koko, Hi...",63,0:02:18,0:00:02
4,8/21/2020,HH08212020-2,0:38:03,,Nina,yes,09-03-2020 (v1); 01-25-2021 (v2),Nina,Sharon,"Nina, Sharon",Alphabet book,"Text on leopard (muzu) from Alphabet M book, s...",158,0:04:47,0:00:01


In [34]:
df['cat_list'] = df['Category']\
    .str.split('+')\
    .apply(lambda l: [s.strip().lower() for s in l])
    
df['cat_list'].astype(str).value_counts()

['paradigms']                     63
['syntax']                        18
['wordlists']                     12
['paradigms', 'story']             5
['alphabet book', 'paradigms']     5
['story', 'paradigms']             3
['tonal elicitation']              3
['story', 'tonal elicitation']     2
['alphabet book']                  2
['alphabet book', 'syntax']        2
['paradigms', 'alphabet book']     1
['paradigms', 'syntax']            1
['phrases']                        1
['paradigms', 'phrases']           1
['story', 'wordlists']             1
['wordlists', 'syntax']            1
['story', 'syntax']                1
Name: cat_list, dtype: int64

In [35]:
utterance_cats = defaultdict(lambda:0)
recording_cats = defaultdict(lambda:0)

def count_utterance_categories(row):
    cats = row['cat_list']
    num_utts = row['num_utterances']
    for cat in cats:
        utterance_cats[cat] += num_utts/len(cats)
        recording_cats[cat] += 1/len(cats)

df.apply(count_utterance_categories, axis=1)
utterance_cats

defaultdict(<function __main__.<lambda>()>,
            {'paradigms': 10326.0,
             'phrases': 162.0,
             'alphabet book': 852.0,
             'syntax': 1660.5,
             'story': 992.5,
             'wordlists': 2458.0,
             'tonal elicitation': 704.0})